# Import Libraries

In [54]:
import os
from langchain_openai import OpenAIEmbeddings
import hashlib
from pinecone import Pinecone
from datetime import date
from transcript2 import sample_transcript2
import time

# API Keys

In [48]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

# Initialization

In [49]:
organization_name = input("Input organization name")
meeting_title = input("Input meeting title")

# Pinecone Initialization
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(organization_name)

# OpenAI Initialization
EMBEDDINGS = OpenAIEmbeddings(model='text-embedding-3-small', openai_api_key=OPENAI_API_KEY)

### Sample Transcript

In [ ]:
sample_transcript1 = """Czech: Hello my name is Czech.     
Gian: Hello my name is Gian.
Shaundyl: Hello my name is Shaundyl.
Czech (Team Lead): Alright, everyone, thanks for joining today’s meeting. We have about 10 minutes to go over the final details before the product launch. Let's start with the progress update. Bob, how are we doing on the development front?
Shaundyl (Developer): Things are looking good. We’ve implemented all the major features. I’m currently working on the final round of bug fixes. I should be done with it by tomorrow, but I need the QA team to give it another pass afterward.
Czech: Great to hear that. Gian, how are we looking on the project timeline? Any changes or concerns?
Gian (Project Manager): We’re on track, but barely. The marketing materials were delayed by two days, but I managed to align the social media schedule to compensate for the delay. As long as the development and testing stay on track, we should meet the launch date. We might want to allocate some buffer time for any last-minute issues though.
Czech: Makes sense. Shaundyl, do you think we’ll have time for a final round of testing before we push live?
Shaundyl: We should. I’ll aim to finish bug fixes by midday tomorrow. If QA can start immediately after, we’ll have 24 hours for testing before the go-live. I’ll stay available for any hotfixes, just in case.
Czech: Perfect. Gian, can you make sure the QA team is on standby for tomorrow afternoon?
Gian: Absolutely. I’ll notify them as soon as we’re done here. I’ll also double-check the launch checklist to make sure nothing’s been missed.
Czech: Great. And one last thing – how are we handling customer support on launch day? Any special preparations?
Gian: We’ve set up a dedicated support channel for the product and briefed the customer support team on the common issues we’re anticipating. We’ll also monitor social media for any unexpected feedback.
Czech: Sounds like we’re in good shape. Thanks, everyone. Let’s aim to regroup tomorrow for a final status check. Anything else before we wrap up?
Shaundyl: Nothing from my side. I’ll update you if any blockers come up.
Gian: I’m all set. Let’s get this done!
Czech: Alright then, thanks again! Talk tomorrow.
"""

# Chunking

### Original Chunking

In [56]:
def chunk_text(text, max_chunk_size=500):
    # Ensure each text ends with a newline to correctly split sentences
    if not text.endswith("\n"):
        text += "\n"

    # Split text into sentence
    sentences = text.split("\n")
    chunks = []
    current_chunk = ""

    # Iterate over sentence and assemble chunks
    for sentence in sentences:
        # Check if adding the current sentence exceeds the maximum chunk size
        if (len(current_chunk) + len(sentences) + 2 > max_chunk_size and current_chunk):
            # Add the current chunk to the list and start a new chunk
            chunks.append(current_chunk.strip())
            current_chunk = ""
        # Add the current sentence to the current chunk
        current_chunk += sentence.strip() + "\n"
    # Add any remaining text as the last chunk
    if (current_chunk):
        chunks.append(current_chunk.strip())

    return chunks # type: list[str]

chunked_text = chunk_text(text=sample_transcript2)
print("\n".join(f"Chunk {i+1}:\n{chunk}" for i, chunk in enumerate(chunked_text)))

Chunk 1:
[00:00:00] John: Good morning, everyone. Thank you for joining today's kickoff meeting for our new     software development project. We'll be discussing the project scope, timelines, and
responsibilities. Let's get started with a quick round of introductions. I'll go first. I'm John, the     project manager. I'll be overseeing the project and ensuring we stay on track. Alice, would you     like to go next?
[00:00:20]
Alice: Sure, thanks John. Hi, everyone. I'm Alice, the lead developer. I'll be responsible for the
Chunk 2:
overall architecture and development of the software. Looking forward to working with all of you.
[00:00:35]
Bob: Hi, I'm Bob, the UI/UX designer. I'll be handling the design aspects of the software, making
sure it's user-friendly and visually appealing.
[00:00:45]
Sara: Hello, I'm Sara, the QA analyst. I'll be testing the software to ensure it meets our quality
standards and is free of bugs.
[00:00:55]
John: Great, thank you. Now that we've introduced ourse

### Recursive Chunking

In [50]:
def chunk_text_recursive(text, max_chunk_size=500):
    # Helper function for recursive chunking
    def recursive_chunk(sentences, current_chunk=""):
        # Base case: if no sentences are left, return the current chunk
        if not sentences:
            return [current_chunk.strip()] if current_chunk.strip() else []

        # Extract the next sentence
        sentence = sentences[0]
        remaining_sentences = sentences[1:]

        # Check if adding the current sentence exceeds the max_chunk_size
        if len(current_chunk) + len(sentence) + 1 > max_chunk_size:
            # Return the current chunk and continue with the next sentences
            return [current_chunk.strip()] + recursive_chunk(remaining_sentences, "")
        else:
            # Add the current sentence and continue recursively
            return recursive_chunk(remaining_sentences, current_chunk + sentence.strip() + "\n")

    # Ensure each text ends with a newline for sentence splitting
    if not text.endswith("\n"):
        text += "\n"

    # Split text into sentences by newline and filter out empty sentences
    sentences = [sentence for sentence in text.split("\n") if sentence.strip()]

    # Start recursive chunking
    return recursive_chunk(sentences)

chunked_text = chunk_text_recursive(text=sample_transcript2)
print("\n".join(f"Chunk {i+1}:\n{chunk}" for i, chunk in enumerate(chunked_text)))

Chunk 1:
[00:00:00] John: Good morning, everyone. Thank you for joining today's kickoff meeting for our new     software development project. We'll be discussing the project scope, timelines, and
responsibilities. Let's get started with a quick round of introductions. I'll go first. I'm John, the     project manager. I'll be overseeing the project and ensuring we stay on track. Alice, would you     like to go next?
[00:00:20]
Chunk 2:
overall architecture and development of the software. Looking forward to working with all of you.
[00:00:35]
Bob: Hi, I'm Bob, the UI/UX designer. I'll be handling the design aspects of the software, making
sure it's user-friendly and visually appealing.
[00:00:45]
Sara: Hello, I'm Sara, the QA analyst. I'll be testing the software to ensure it meets our quality
standards and is free of bugs.
[00:00:55]
Chunk 3:
Our goal is to develop a new customer management system for our client. The system should
allow users to manage customer data, track interactions

# Generate Embeddings

In [51]:
def generate_embeddings(texts):
    """
    Generate embeddings for a list of text.
    """
    embedded = EMBEDDINGS.embed_documents(texts)

    print("Generating embeddings: Done!")
    return embedded

chunked_text_embeddings = generate_embeddings(texts=chunked_text)
print(chunked_text_embeddings)

Generating embeddings: Done!
[[0.013034100644290447, 0.04776821658015251, 0.040253061801195145, -0.019609862938523293, -0.03191593289375305, -0.030718205496668816, 0.014713268727064133, 0.014255314134061337, 0.017684103921055794, -0.019762514159083366, 0.03891442343592644, -0.03771669417619705, -0.03325457125902176, -0.03708260506391525, 0.026655323803424835, -0.0011338787153363228, -0.06711974740028381, -2.974596827698406e-05, -0.03990078717470169, 0.00831364281475544, 0.006100194063037634, 0.01328069157898426, -0.005885894410312176, -0.00577140599489212, -0.026984110474586487, -0.012587888166308403, -0.04156821221113205, 0.021817440167069435, 0.020654939115047455, -0.01109072845429182, -0.013597737066447735, -0.02320304699242115, -0.0028181839734315872, 0.015969708561897278, 0.003478695871308446, -0.021570850163698196, -0.004600098356604576, -0.005639303475618362, -0.026138655841350555, -0.006998490076512098, 0.005448488984256983, -0.030201537534594536, 0.018517816439270973, -0.04464

# Combine Vector and Text

In [ ]:
def generate_short_id(content):
    """
    Generate a short ID based on the content using SHA-256 hash.
    """
    # Generate short id
    hash_obj = hashlib.sha256()
    hash_obj.update(content.encode("utf-8"))

    print("Generating short id: Done!")
    return hash_obj.hexdigest()

def combine_vector_and_text(texts, meeting_title, text_embeddings):
    """
    Process a list of texts along with their embeddings.
    """
    # Date Today
    today = str(date.today())
    
    data_with_metadata = []

    # Creates list that contains id, values, and metadata
    for doc_text, embedding in zip(texts, text_embeddings):
        if not isinstance(doc_text, str):
            doc_text = str(doc_text)

        if not isinstance(meeting_title, str):
            meeting_title = str(meeting_title)

        if not isinstance(today, str):
            today = str(today)

        text_id = generate_short_id(doc_text)
        data_item = {
            "id": text_id,
            "values": embedding,
            "metadata": {"text": doc_text, "title": meeting_title, "date": today},
        }

        data_with_metadata.append(data_item)

    print("Combining vector and text: Done!")
    return data_with_metadata

data_with_meta_data = combine_vector_and_text(texts=chunked_text, meeting_title=meeting_title, text_embeddings=chunked_text_embeddings)
print(data_with_meta_data)

Generating short id: Done!
Generating short id: Done!
Generating short id: Done!
Generating short id: Done!
Generating short id: Done!
Generating short id: Done!
Generating short id: Done!
Combining vector and text: Done!
[{'id': 'e93334d3494ce6084f72308a5e6905227e7f8a5f8c59169eba8afe5a9de03adb', 'values': [0.013034100644290447, 0.04776821658015251, 0.040253061801195145, -0.019609862938523293, -0.03191593289375305, -0.030718205496668816, 0.014713268727064133, 0.014255314134061337, 0.017684103921055794, -0.019762514159083366, 0.03891442343592644, -0.03771669417619705, -0.03325457125902176, -0.03708260506391525, 0.026655323803424835, -0.0011338787153363228, -0.06711974740028381, -2.974596827698406e-05, -0.03990078717470169, 0.00831364281475544, 0.006100194063037634, 0.01328069157898426, -0.005885894410312176, -0.00577140599489212, -0.026984110474586487, -0.012587888166308403, -0.04156821221113205, 0.021817440167069435, 0.020654939115047455, -0.01109072845429182, -0.013597737066447735, -0

# Upsert to Pinecone

In [ ]:
def upsert_data_to_pinecone(data_with_metadata, namespace):
    """
    Upsert data with metadata into a Pinecone index.
    """
    index.upsert(vectors=data_with_metadata, namespace=namespace)
    print("Upserting vectors to Pinecone: Done!")

upsert_data_to_pinecone(data_with_metadata=data_with_meta_data, namespace=meeting_title)
time.sleep(2)
index.describe_index_stats()

Upserting vectors to Pinecone: Done!


{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'Project Meeting': {'vector_count': 4}},
 'total_vector_count': 4}

# MAIN

In [ ]:
def store_to_pinecone(texts, meeting_title):
    today = str(date.today()) # INITIALIZATION FOR DATE (DYNAMIC) BASED ON STORING

    chunked_text = chunk_text(text=texts)
    chunked_text_embeddings = generate_embeddings(texts=chunked_text)
    data_with_meta_data = combine_vector_and_text(texts=chunked_text, meeting_title=meeting_title, date=today,  text_embeddings=chunked_text_embeddings)
    upsert_data_to_pinecone(data_with_metadata=data_with_meta_data, namespace_name=meeting_title)

store_to_pinecone(texts=sample_transcript, meeting_title=index,)

Generating embeddings: Done!
Generating short id: Done!
Generating short id: Done!
Generating short id: Done!
Generating short id: Done!
Generating short id: Done!
Combining vector and text: Done!


TypeError: upsert_data_to_pinecone() got an unexpected keyword argument 'namespace_name'